# Лабораторная работа 2. Метод ближайших соседей и решающие деревья.

ФИО: Попов Артём Сергеевич

Группа: 317

In [19]:
import numpy as np
import pandas as pd

Все эксперименты в этой лабораторной работе предлагается проводить на данных соревнования Amazon Employee Access Challenge: https://www.kaggle.com/c/amazon-employee-access-challenge

В данной задаче предлагается предсказать, будет ли одобрен запрос сотрудника на получение доступа к тому или иному ресурсу. Все признаки являются категориальными.

Для удобства данные можно загрузить по ссылке: https://www.dropbox.com/s/q6fbs1vvhd5kvek/amazon.csv

Сразу прочитаем данные и создадим разбиение на обучение и контроль:

In [2]:
data = pd.read_csv('data/amazon.csv')
data.head()

,ACTION,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE
0,1,39353,85475,117961,118300,123472,117905,117906,290919,117908
1,1,17183,1540,117961,118343,123125,118536,118536,308574,118539
2,1,36724,14457,118219,118220,117884,117879,267952,19721,117880
3,1,36135,5396,117961,118343,119993,118321,240983,290919,118322
4,1,42680,5905,117929,117930,119569,119323,123932,19793,119325


In [3]:
data.shape

(32769, 10)

In [4]:
# доля положительных примеров
data.ACTION.mean()

0.94210992096188473

In [5]:
# число значений у признаков
for col_name in data.columns:
    print col_name, len(data[col_name].unique())

ACTION 2
RESOURCE 7518
MGR_ID 4243
ROLE_ROLLUP_1 128
ROLE_ROLLUP_2 177
ROLE_DEPTNAME 449
ROLE_TITLE 343
ROLE_FAMILY_DESC 2358
ROLE_FAMILY 67
ROLE_CODE 343


In [78]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.iloc[:, 1:], data.iloc[:, 0],
                                                    test_size=0.3, random_state=241)

In [7]:
X_test.shape

(9831, 9)

## Часть 1: kNN и категориальные признаки

#### 1. Реализуйте три функции расстояния на категориальных признаках, которые обсуждались на втором семинаре. Реализуйте самостоятельно метод k ближайших соседей, который будет уметь работать с этими функциями расстояния. Подсчитайте для каждой из них качество на тестовой выборке `X_test` при числе соседей $k = 10$. Метрика качества — AUC-ROC.

Какая функция расстояния оказалась лучшей?

Все метрики были реализованы так, чтобы на выходе оказывалась матрица расстояний: в ij ячейке расстояние от i-ого объекта первой матрицы до j-го объекта второй матрицы.

1 метрика

In [8]:
def dis_1_matrix(X, Z):
    X_1 = np.array(X)
    Z_1 = np.array(Z)
    return np.sum(X_1[:, np.newaxis] != Z_1[np.newaxis, :], axis=2)

In [9]:
import collections # для defaultdict

Реализация вспомогательных функций f, p и p2, описанных на семинаре.

In [10]:
def create_f_list(data):
    f = list()
    for j in range(0, data.shape[1]):
        column = data.iloc[:, j]
        f.append(dict(column.value_counts()))        
    return f

def create_p_list(f, l):
    p1 = list()
    for i in range(len(f)):
        p1.append(collections.defaultdict(int))
        for (key, value) in f[i].items():
            p1[i][key] = value * 1.0 / l
    return p1

def create_p2_list(f, l):
    p2 = list()
    for i in range(len(f)):
        p2.append(collections.defaultdict(int))
        for (key, value) in f[i].items():
            p2[i][key] = value * (value - 1) * 1.0 / (l * (l - 1))
    return p2

def create_log_f(f):
    logf = list()
    for i in range(len(f)):
        logf.append(collections.defaultdict(int))
        for (key, value) in f[i].items():
            logf[i][key] = np.log(value * 1.0)
    return logf

2 метрика

In [12]:
def dis_2_matrix(X, Z):
    f = create_f_list(Z)
    p = create_p_list(f, Z.shape[0])
    p2 = create_p2_list(f, Z.shape[0])
    l_np_arr_p2 = [np.array(item.values()) for item in p2]
    l_np_arr = [np.array(item.values()) for item in p]
    
    res = np.zeros((X.shape[0], Z.shape[0]))
    for i, x in enumerate(np.array(X)):
        for j, z in enumerate(np.array(Z)):
            for k in range(X.shape[1]):
                if (x[k] == z[k]):
                    res[i, j] += np.sum(l_np_arr_p2[k][l_np_arr[k] <= p[k].get(x[k], 0)])
    return res + dis_1_matrix(X, Z)

3 метрика

In [13]:
def dis_3_matrix(X, Z):
    f = create_f_list(Z)
    logf = create_log_f(f)
    res = np.zeros((X.shape[0], Z.shape[0]))
    for i, x in enumerate(np.array(X)):
        for j, z in enumerate(np.array(Z)):
            for k in range(X.shape[1]):
                if (x[k] != z[k]):
                    res[i, j] += logf[k].get(x[k], 0) * logf[k].get(z[k], 0)
    return res

Подключение необходимых библиотек

In [14]:
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cross_validation import KFold

Функция, реализующая метод KNN. Внутри функции происходит подсчёт матрицы расстояний с помощью соответствующей метрики. В функции реализовано нахождение наилучшего k для тестовой выборки.

In [15]:
def my_super_KNN(X_train, y_train, X_test, y_test, my_metric):
    matr_dis = my_metric(X_test, X_train)
    best = 0.0
    k_best = 1
    for k in range(1, 25):
        ind = matr_dis.argsort(axis=1)[:,:k]
        num_class = np.array(y_train.as_matrix()[ind])
        ans = np.sum(num_class, axis=1) * 1.0 / k
        roc = roc_auc_score(y_test, ans)
        if (k == 10):
            print('if k == 10:')
            print(roc)
        if (best < roc):
            best = roc
            k_best = k
    print('the best k == ' + str(k_best))
    print(best)
    return (k_best, best)

Нахождение точности для k=10 и нахождение наилучшего k:

In [16]:
roc1 = my_super_KNN(X_train, y_train, X_test, y_test, dis_1_matrix) 

if k == 10:
0.83088009598
the best k == 11
0.831201958708


In [17]:
roc2 = my_super_KNN(X_train, y_train, X_test, y_test, dis_2_matrix) 

if k == 10:
0.832873632128
the best k == 7
0.836160998935


In [18]:
roc3 = my_super_KNN(X_train, y_train, X_test, y_test, dis_3_matrix) 

if k == 10:
0.817053027738
the best k == 11
0.817141176073


#### 2 (бонус). Подберите лучшее (на тестовой выборке) число соседей $k$ для каждой из функций расстояния. Какое наилучшее качество удалось достичь?

Это задание уже выполнено в предыдущем пункте. Наилучшее качество при второй метрике и k=11: 0.8361

#### 3. Реализуйте счетчики (http://blogs.technet.com/b/machinelearning/archive/2015/02/17/big-learning-made-easy-with-counts.aspx), которые заменят категориальные признаки на вещественные.

А именно, каждый категориальный признак нужно заменить на три: 
1. Число `counts` объектов в обучающей выборке с таким же значением признака.
2. Число `clicks` объектов первого класса ($y = 1$) в обучающей выборке с таким же значением признака.
3. Сглаженное отношение двух предыдущих величин: (`clicks` + 1) / (`counts` + 2).

Поскольку признаки, содержащие информацию о целевой переменной, могут привести к переобучению, может оказаться полезным сделать *фолдинг*: разбить обучающую выборку на $n$ частей, и для $i$-й части считать `counts` и `clicks` по всем остальным частям. Для тестовой выборки используются счетчики, посчитанный по всей обучающей выборке. Реализуйте и такой вариант. Можно использовать $n = 3$.

#### Посчитайте на тесте AUC-ROC метода $k$ ближайших соседей с евклидовой метрикой для выборки, где категориальные признаки заменены на счетчики. Сравните по AUC-ROC два варианта формирования выборки — с фолдингом и без. Не забудьте подобрать наилучшее число соседей $k$.

Функция add_counts, генерирующая обучающую и тестовую выборку с новыми признаками:

In [20]:
def add_counts(X_train, y_train, X_test, y_test):
    f = create_f_list(X_train)
    X_new_train = pd.DataFrame()
    X_new_test = pd.DataFrame()
    Pos_examples = X_train.iloc[np.where(y_train == 1)[0]]
    f_pos = create_f_list(Pos_examples)
        
    for j in range(X_train.shape[1]):
        name_col = str(X_train.columns.values[j])
        X_new_train['count_' + name_col] = X_train[name_col].apply(lambda x: f[j].get(x, 0))
        X_new_train['clicks_' + name_col] = X_train[name_col].apply(lambda x: f_pos[j].get(x, 0))
        X_new_train['p_' + name_col] = (X_new_train['clicks_' + name_col] + 1) * 1.0 / \
                                       (X_new_train['count_' + name_col] + 2)
        X_new_test['count_' + name_col] = X_test[name_col].apply(lambda x: f[j].get(x, 0))
        X_new_test['clicks_' + name_col] = X_test[name_col].apply(lambda x: f_pos[j].get(x, 0))
        X_new_test['p_' + name_col] = (X_new_test['clicks_' + name_col] + 1) * 1.0 / \
                                      (X_new_test['count_' + name_col] + 2)
            
    return X_new_train, X_new_test

Функция для реализации KNN с счётчиками:

In [21]:
def KNN_with_counts(X_train, y_train, X_test, y_test, k):
    X_new_train, X_new_test = add_counts(X_train, y_train, X_test, y_test)
    scaler = StandardScaler()
    X_new_train = scaler.fit_transform(X_new_train)
    X_new_test = scaler.transform(X_new_test)
    clf1 = KNeighborsClassifier(n_neighbors=k)
    clf1.fit(X_new_train, y_train)
    return (roc_auc_score(y_test, clf1.predict_proba(X_new_test)[:, 1]))

Нахождение наилучшего k:

In [119]:
best = 0
k_best = 0
for k in range(1, 25):
    res = KNN_with_counts(X_train, y_train, X_test, y_test, k)
    if best < res:
        best = res
        k_best = k

print(k_best, best)

(19, 0.80185575158451372)


Наилучшая точность при k = 19:

0.80185

Функция, реализующая кросс-валидацию для KNN с счётчиками:

In [22]:
def K_fold_with_params(data, k):
    X_all_train = data.iloc[:, 1:]
    y_all_train = data.iloc[:, 0]
    
    pos = 0
    nf = 4
    
    kf = KFold(X_all_train.shape[0], n_folds=nf)
    
    for (ind_train, ind_test) in kf:
        X_train = X_all_train.iloc[ind_train]
        X_test = X_all_train.iloc[ind_test]
        y_train = y_all_train.iloc[ind_train]
        y_test = y_all_train.iloc[ind_test]
        
        pos += KNN_with_counts(X_train, y_train, X_test, y_test, k)
    return pos / nf

Нахождение наилучшего k:

In [122]:
best = 0
k_best = 0
for k in range(1, 20):
    res = K_fold_with_params(data, k)
    if best < res:
        best = res
        k_best = k

print(k_best, best)

(19, 0.80874517332017737)


Наилучшая точность при k = 19 (как и на тестовой выборке):

0.808745

#### 4. Добавьте в исходную выборку парные признаки — то есть для каждой пары $f_i$, $f_j$ исходных категориальных признаков добавьте новый категориальный признак $f_{ij}$, значение которого является конкатенацией значений $f_i$ и $f_j$. Посчитайте счетчики для этой выборки, найдите качество метода $k$ ближайших соседей с наилучшим $k$ (с фолдингом и без).

Функция, добавляющая парные признаки:

In [23]:
def pair_counts(X_train, X_test):
    X_new_train = pd.DataFrame()
    X_new_test = pd.DataFrame()
    for i in range(X_train.shape[1]):
        name_col_i = str(X_train.columns.values[i])
        X_new_train[name_col_i] = X_train[name_col_i]
        X_new_test[name_col_i] = X_test[name_col_i]
        for j in range(X_train.shape[1]):
            if (j != i):
                name_col_j = str(X_train.columns.values[j])
                temp1 = X_train[name_col_i].apply(lambda x: str(x))
                temp2 = X_train[name_col_j].apply(lambda x: str(x))
                temp3 = X_test[name_col_i].apply(lambda x: str(x))
                temp4 = X_test[name_col_j].apply(lambda x: str(x))
                
                X_new_train['concat_' + str(i) + str(j)] = temp1 + temp2
                X_new_test['concat_' + str(i) + str(j)] = temp3 + temp4
    return X_new_train, X_new_test
    

In [130]:
X_new_train, X_new_test = pair_counts(X_train, X_test)

Нахождение наилучшего k:

In [132]:
best = 0
k_best = 0
for k in range(1, 25):
    res = KNN_with_counts(X_new_train, y_train, X_new_test, y_test, k)
    if best < res:
        best = res
        k_best = k
        
print(k_best, best)

(23, 0.82933718562224279)


Наилучший результат среди k от 1 до 25 при k=23: 0.82933

In [24]:
def K_fold_with_pairs(data, k):
    X_all_train = data.iloc[:, 1:]
    y_all_train = data.iloc[:, 0]
    
    pos = 0
    nf = 4
    
    kf = KFold(X_all_train.shape[0], n_folds=nf)
    
    for (ind_train, ind_test) in kf:
        X_train = X_all_train.iloc[ind_train]
        X_test = X_all_train.iloc[ind_test]
        y_train = y_all_train.iloc[ind_train]
        y_test = y_all_train.iloc[ind_test]
        X_new_train, X_new_test = pair_counts(X_train, X_test)
        pos += KNN_with_counts(X_new_train, y_train, X_new_test, y_test, k)
    return pos / nf

Нахождение наилучшего k по кросс-валидации с 4 фолдами:

In [239]:
best = 0
k_best = 0

for k in range(1, 20):
    res = K_fold_with_pairs(data, k)
    if best < res:
        best = res
        k_best = k
        
print(k_best, res)   

(19, 0.8286647308046251)


Лучшее качество при k=19 (проверялись значения от 1 до 19)

0.826866

## Часть 2: Решающие деревья и леса

#### 1. Возьмите из предыдущей части выборку с парными признаками, преобразованную с помощью счетчиков без фолдинга. Настройте решающее дерево, подобрав оптимальные значения параметров `max_depth` и `min_samples_leaf`. Какой наилучший AUC-ROC на контроле удалось получить?

In [79]:
X_new_train, X_new_test = pair_counts(X_train, X_test)
X_new_train, X_new_test = add_counts(X_new_train, y_train, X_new_test, y_test)

In [25]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.grid_search import GridSearchCV

In [207]:
kf = KFold(X_new_train.shape[0], shuffle=True, random_state=241, n_folds=5)
gs = GridSearchCV(DecisionTreeClassifier(random_state=241),
                   param_grid={'max_depth': [2, 4, 6, 8, 10, 20, 50], 
                              'min_samples_leaf': range(1, 20)},
                   cv=kf,
                   scoring='roc_auc')

gs.fit(X_new_train, y_train)

GridSearchCV(cv=sklearn.cross_validation.KFold(n=22938, n_folds=5, shuffle=True, random_state=241),
       error_score='raise',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            random_state=241, splitter='best'),
       fit_params={}, iid=True, loss_func=None, n_jobs=1,
       param_grid={'max_depth': [2, 4, 6, 8, 10, 20, 50], 'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]},
       pre_dispatch='2*n_jobs', refit=True, score_func=None,
       scoring='roc_auc', verbose=0)

Лучший результат с обучающей выборкой:

In [208]:
gs.best_score_

0.99955563868376263

In [209]:
for z in gs.grid_scores_:
    if z.mean_validation_score == gs.best_score_:
        print z

mean: 0.99956, std: 0.00079, params: {'max_depth': 4, 'min_samples_leaf': 18}


С теми же параметрами на тестовой:

In [210]:
ds = DecisionTreeClassifier(random_state=241, max_depth=4, min_samples_leaf=18)
ds.fit(X_new_train, y_train)
roc_auc_score(y_test, ds.predict_proba(X_new_test)[:, 1])

0.57868977608705507

Попробуем поперебирать другие параметры:

In [211]:
gs.grid_scores_

[mean: 0.99054, std: 0.00571, params: {'max_depth': 2, 'min_samples_leaf': 1},
 mean: 0.99054, std: 0.00571, params: {'max_depth': 2, 'min_samples_leaf': 2},
 mean: 0.99054, std: 0.00571, params: {'max_depth': 2, 'min_samples_leaf': 3},
 mean: 0.99054, std: 0.00571, params: {'max_depth': 2, 'min_samples_leaf': 4},
 mean: 0.99054, std: 0.00571, params: {'max_depth': 2, 'min_samples_leaf': 5},
 mean: 0.99054, std: 0.00571, params: {'max_depth': 2, 'min_samples_leaf': 6},
 mean: 0.99054, std: 0.00570, params: {'max_depth': 2, 'min_samples_leaf': 7},
 mean: 0.99054, std: 0.00570, params: {'max_depth': 2, 'min_samples_leaf': 8},
 mean: 0.99053, std: 0.00569, params: {'max_depth': 2, 'min_samples_leaf': 9},
 mean: 0.99053, std: 0.00569, params: {'max_depth': 2, 'min_samples_leaf': 10},
 mean: 0.99053, std: 0.00569, params: {'max_depth': 2, 'min_samples_leaf': 11},
 mean: 0.99053, std: 0.00569, params: {'max_depth': 2, 'min_samples_leaf': 12},
 mean: 0.99053, std: 0.00569, params: {'max_depth

In [220]:
ds = DecisionTreeClassifier(random_state=241, max_depth=10, min_samples_leaf=10)
ds.fit(X_new_train, y_train)
roc_auc_score(y_test, ds.predict_proba(X_new_test)[:, 1])

0.57883570156729724

Улучшение незначительно

#### 2. Настройте случайный лес, подобрав оптимальное число деревьев `n_estimators`. Какое качество на тестовой выборке он дает?

In [80]:
from sklearn.ensemble import RandomForestClassifier

In [82]:
best = 0
n_best = 0

for n in range(25, 200, 25):
    cl = RandomForestClassifier(n_estimators=n)
    cl.fit(X_new_train, y_train)
    res = roc_auc_score(y_test, cl.predict_proba(X_new_test)[:, 1])
    if res > best:
        best = res
        n_best = n

print(n_best, best)

(150, 0.71747272523382211)


Лучший результат при 150 деревьях 

0.717

#### 3. Возьмите выборку с парными признаками, для которой счетчики посчитаны с фолдингом. Обучите на ней случайный лес, подобрав число деревьев. Какое качество на тестовой выборке он дает? Чем вы можете объяснить изменение результата по сравнению с предыдущим пунктом?

In [83]:
data_super = pd.DataFrame()
y_super = np.array((1))
pos = 0
nf = 4

data1 = data.iloc[:, 1:]
y = data.iloc[:, 0]

kf = KFold(data.shape[0], n_folds=nf)
    
for (ind_train, ind_test) in kf:
    X_train1 = data1.iloc[ind_train]
    X_test1 = data1.iloc[ind_test]
    y_train1 = y.iloc[ind_train]
    y_test1 = y.iloc[ind_test]
    X_new_train, X_new_test = pair_counts(X_train1, X_test1)
    X_new_train, X_new_test = add_counts(X_new_train, y_train1, X_new_test, y_test1)
    data_super = pd.concat([data_super, X_new_test], axis=0)
    y_super = np.hstack((y_super, y_test1))

y_super = y_super[1:]

In [84]:
X_train, X_test, y_train, y_test = train_test_split(data_super, y_super,
                                                    test_size=0.3, random_state=241)

In [86]:
best = 0
n_best = 0

for n in range(50, 200, 25):
    cl = RandomForestClassifier(n_estimators=n)
    cl.fit(X_train, y_train)
    res = roc_auc_score(y_test, cl.predict_proba(X_test)[:, 1])
    if res > best:
        best = res
        n_best = n

print(n_best, best)

(125, 0.88002939715524398)


Лучший результат при 125 деревьяв:
    
0.88

Можно сделать вывод, что подсчёт счётчиков с фолдингом не обязателен для KNN, но нужен для лесов, т.к. счётчики без фолдинга быстро приводят к переобучению.